In [ ]:
wd<-"/Seurat/reg.CC/WT_replace_v2/res02_220310/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
#library(hdf5r)
library(png)
#library(ggalluvial)
#library(tidyverse)
library(clustree)
library(RCTD)
library(Matrix)
#library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
#library(pheatmap)
library(circlize)
library(igraph)
#library(ComplexHeatmap)
library(stringr)
#library(ComplexHeatmap)
library(cowplot)
library(biomaRt)
#library(scClustViz)
library(scales)
library(colorspace)
library(biomaRt)
library(clusterProfiler)
options(connectionObserver = NULL)
library(reshape2)
library(pheatmap)
library("readxl")

### Astrocyte migration marker score -220612
#### positive regulation of cell migration score -220620

In [3]:
wd<-"Astrocyte/"
if(!dir.exists(wd))
    dir.create(wd)
#setwd(wd)
#getwd()

In [4]:
source("/home/jovyan/zxli_SCI/script/self_function/save_pheatmap_pdf.R")

In [ ]:
cm<-read_excel("/home/jovyan/zxli_SCI/result/Seurat/sc_Astrocyte_recluster/lee/classifyBySpatialMarker/GO_term_positiveregulationofcellmigration_20220620_052409.xlsx")
head(cm)
cmgenes<-unique(cm$Symbol)
length(cmgenes)
cmgenes<-intersect(cmgenes,rownames(da@assays$SCT@data))
length(cmgenes)


In [23]:
da<-AddModuleScore(da,features = list(cmgenes),name = "positive_regulation_of_cell_migration")

In [ ]:
da$pos_mig_bi<-as.character(ifelse(da$positive_regulation_of_cell_migration1>0.07204,1,0))
table(da$pos_mig_bi)

In [ ]:
#Mki67
da$Mki67<-da@assays$SCT@data["Mki67",]
#head(da$Mki67)
summary(da$Mki67)
#
#da$Mki67_bi<-as.character(ifelse(da$Mki67>0,1,0))
#table(da$Mki67_bi)

In [25]:
meta<-da@meta.data
write.csv(meta,"WT.Mki67.migration_score.meta.csv")

In [ ]:
options(repr.plot.width=20,repr.plot.height=16)
ggplot(da@meta.data,aes(x.ad,y.ad))+geom_point(size=1.6,shape=21,stroke=0.15,aes(fill= positive_regulation_of_cell_migration1))+
    #scale_fill_gradientn(colours = colorRampPalette(rev(brewer.pal(n = 7,name = "Spectral")))(50))+
    scale_fill_gradientn(colours = c('#034f84','#92a8d1',#'#b8a9c9', 
    
        '#d6d4e0',#"#deeaee",
    #"#f0f0f0",
    #'#fff2df',
       
        #"#eeac99",
        '#f4a688',
        "#ED797B",                             
        #"#f7786b",                             
        
        "#d64161",
        "#c94c4c"#,
        #"#c83349"
                                    ))+
    xlab(paste0("")) +
    ylab(paste0("")) + 
    theme(panel.background = element_blank(),
          panel.grid.major =element_blank(), 
          panel.grid.minor = element_blank(),
          axis.text.x = element_blank(),#element_text(size = 15,colour = "black"),
          axis.text.y = element_blank()#element_text(size = 20,colour = "black")
          # axis.ticks.x = element_blank(),
    )
#ggsave("WT.merge.regress_CC.nC.mt.ident_positive_regulation_of_cell_migration.spatial_manually_220701.png",width=20,height=16,dpi=400)

In [ ]:
### calculate the colocalization pval of cell type each timepoint -220627
ct_df<-read.csv("/home/jovyan/zxli_SCI/result/RCTD/testC5/WT_all/fcreg1_220503/16celltypes.dec_conf.nor.meta.csv")
rownames(ct_df)<-ct_df[,1]
ct_df<-ct_df[,-1]
head(ct_df)

In [ ]:
ct<-colnames(ct_df)[46:61]
ct

In [ ]:
# remove Svep1 and Dendritic
ct<-ct[!ct%in%c("Astro.Svep1","Dendritic")]
ct

In [ ]:
wt_df<-ct_df[,ct]
dim(wt_df)

In [ ]:
wt_df_bi<-wt_df
wt_df_bi<-apply(wt_df_bi,2,function(x) (ifelse(x>0,1,0)))
head(wt_df_bi)

In [ ]:
me<-cbind(da@meta.data,wt_df_bi)
dim(me)

In [42]:
## Fisher test and jaccard simmilarity
pval_df<-data.frame(row.names = unique(me$time))
spot_n<-data.frame(row.names = unique(me$time))
jc_df<-data.frame(row.names = unique(me$time))
#pcc_df<-data.frame(row.names = unique(me$time))
for(i in unique(me$time)){
    temp_me<-me[me$time==i,]
    cells<-rownames(temp_me)
    temp_wt<-wt_df_bi[cells,ct]
    ## gene normalized express
    #cc1<-temp_me[cells,]
    ## Mki67 positive spots
    #pp1<-temp_me[,"pos_mig_bi"]
    pp1<-temp_me[,"Mki67_bi"]
    lr_ind<-rownames(temp_me)[pp1==1]
    for(j in ct){
        if(sum(temp_wt[,j])==0){
            pval_df[i,j]<-1
            spot_n[i,j]<-0
            jc_df[i,j]<-0
        }
        else{
            # unbinary wt
            #cc2<-wt_df[cells,j]
            # binary wt
            pp2<-temp_me[,j]
            cp_ind<-rownames(temp_me)[pp2==1]
            # pcc<-cor(pp1,pp2)
            pp1<-factor(pp1,levels=c(0,1))
            pp2<-factor(pp2,levels=c(0,1))
            f_df<-table(pp1,pp2)
            p<-fisher.test(f_df)$p.val
            pval_df[i,j]<-p
            
            n<-f_df[2,2]
            #nn<-f_df[1,1]
            spot_n[i,j]<-n
            
            JC<-length(intersect(cp_ind,lr_ind))/length(unique(c(cp_ind,lr_ind)))
            jc_df[i,j]<-JC
            
            #cc<-cor(cc1,cc2)
            #pcc_df[i,j]<-cc
        }
    }
}

In [ ]:
pval_l<-pval_df
pval_l$time<-rownames(pval_l)
pval_l<-melt(pval_l)
colnames(pval_l)<-c("time","celltype","pval")
pval_l$key<-paste0(pval_l$time,"-",pval_l$celltype)
head(pval_l)

In [ ]:
nspot_l<-spot_n
nspot_l$time<-rownames(nspot_l)
nspot_l<-melt(nspot_l)
colnames(nspot_l)<-c("time","celltype","n_spot")
nspot_l$key<-paste0(nspot_l$time,"-",nspot_l$celltype)
head(nspot_l)

In [ ]:
jc_l<-jc_df
jc_l$time<-rownames(jc_l)
jc_l<-melt(jc_l)
colnames(jc_l)<-c("time","celltype","JC")
jc_l$key<-paste0(jc_l$time,"-",jc_l$celltype)
head(jc_l)

In [ ]:
df_l<-merge(pval_l,nspot_l,by = "key")
dim(df_l)
df_l<-merge(df_l,jc_l,by = "key")
dim(df_l)

In [47]:
write.csv(df_l,paste0(wd,"Mki67_bi_bymean.14celltype.colocalization_eachtime.pval.nspot_220727.csv"))

In [ ]:
df_l<-read.csv("Astrocyte/positive_migration/pos_mig_bi.celltype.colocalization.pval.nspot.csv")
head(df_l)

In [48]:
df_l_ad<-df_l

In [ ]:
pval_ma<-df_l_ad[,c("time","celltype","JC")]
head(pval_ma)
pval_ma<-reshape(pval_ma,idvar = "time",timevar = "celltype",direction = "wide")
rownames(pval_ma)<-pval_ma[,1]
pval_ma<-pval_ma[,-1]
colnames(pval_ma)<-gsub("JC.","",colnames(pval_ma))
pval_ma<-as.matrix(pval_ma[c(4,2,1,3),])
head(pval_ma)

In [ ]:
options(repr.plot.width=7,repr.plot.height=4)
p<-pheatmap(pval_ma,scale = "none",cluster_rows = F,cluster_cols = T#,color = brewer.pal(n = 7,name = "Reds")
           )
#save_pheatmap_pdf(p,"pos_mig_mean_bi.14ct.Fisher_test.pval.jc005.pheatmap_220727.pdf",width = 7,height = 4)

In [ ]:
## remove decreasing cell types: Neuron, Oligodendrocyte
pval_ma_s<-pval_ma[,!colnames(pval_ma)%in% c("Neuron","Oligodendrocyte")]
dim(pval_ma_s)

In [ ]:
options(repr.plot.width=6.3,repr.plot.height=3.8)
p<-pheatmap(pval_ma_s,scale = "none",cluster_rows = F,cluster_cols = T)
save_pheatmap_pdf(p,"Mki67.binary_byMean.jc005.rm_NO.12ct.JC,pheatmap_220727.pdf",width = 6.3,height = 3.8)

In [ ]:
options(repr.plot.width=9,repr.plot.height=7)
p<-FeaturePlot(da,reduction="umap",features = "Cluster1"#,min.cutoff = (-0.03413)
              )+
        theme(#legend.position ="bottom",
            axis.ticks = element_blank(),axis.text = element_blank(),
            axis.line = element_line(size = 0.8,arrow = arrow()) )+
    scale_color_gradientn(colours = c(#"#cfe0e8",
    
                                                     '#b8a9c9', 
    
        '#d6d4e0',"#deeaee",
    #"#f0f0f0",
    #'#fff2df',
       #"#f7786b",
        "#eeac99",
        "#ED797B",
        "#d64161",
        "#c94c4c",
        "#c83349"
    ))
plot(p)
ggsave("Astrocyte/postive_cell_migration.score.data.UMAP.png",width = 9,height = 7,dpi = 300)

In [ ]:
da$time_domain<-paste0(da$time,"-",da$domain_res02)
da$time_dis_domain<-paste0(da$time,"-",da$distance,"-",da$domain_res02)
da$time_pos_domain<-paste0(da$pos,"-",da$domain_res02)
table(da$time_domain)
table(da$time_dis_domain)
table(da$time_pos_domain)

In [ ]:
da$domain2<-ifelse(da$domain_res02=="WM","WM","GM")
da$time_domain2<-paste0(da$time,"-",da$domain2)
da$time_dis_domain2<-paste0(da$time,"-",da$distance,"-",da$domain2)
da$time_pos_domain2<-paste0(da$pos,"-",da$domain2)
table(da$time_domain2)
table(da$time_dis_domain2)
table(da$time_pos_domain2)

In [ ]:
da$time_domain<-factor(da$time_domain,levels = c('WT_sham-WM',
                                                 'WT_3h-WM',
                                                 'WT_24h-WM',
                                                 'WT_72h-WM',
                                                 'WT_sham-VH',
                                                  'WT_3h-VH',
                                                 'WT_24h-VH',
                                                 'WT_72h-VH',
                                                 'WT_sham-MG',
                                                  'WT_3h-MG',
                                                 'WT_24h-MG',
                                                 'WT_72h-MG',
                                                 'WT_sham-DH',
                                                 'WT_3h-DH',
                                                 'WT_24h-DH',
                                                 'WT_72h-DH'))
Idents(da)<-da$time_domain
options(repr.plot.width=5,repr.plot.height=4)
VlnPlot(da,features = "Cluster1",
        pt.size = 0,
        cols = c(rep('#20854EA8',4),rep('#E18727A8',4),rep('#0072B5A8',4),rep('#BC3C29A8',4)))+
    NoLegend()+
    geom_boxplot(width=0.35,fill="white",outlier.color = NA)
    #scale_color_manual(values = c(rep('#20854EA8',4),rep('#E18727A8',4),rep('#0072B5A8',4),rep('#BC3C29A8',4)))
ggsave("Astrocyte/postive_cell_migration.score.in.time_4doamin.vlnplot.png",width = 5,height = 4,dpi = 300)

In [ ]:
da$time_domain2<-factor(da$time_domain2,levels = c('WT_sham-WM',
                                                 'WT_3h-WM',
                                                 'WT_24h-WM',
                                                 'WT_72h-WM',
                                                 'WT_sham-GM',
                                                  'WT_3h-GM',
                                                 'WT_24h-GM',
                                                 'WT_72h-GM'))
Idents(da)<-da$time_domain2
options(repr.plot.width=4,repr.plot.height=4)
VlnPlot(da,features = "Cluster1",
        pt.size = 0,
        cols = c(rep('#20854EA8',4),rep('#0072B5A8',4)))+
    NoLegend()+
    geom_boxplot(width=0.35,fill="white",outlier.color = NA)
    #scale_color_manual(values = c(rep('#20854EA8',4),rep('#E18727A8',4),rep('#0072B5A8',4),rep('#BC3C29A8',4)))
ggsave("Astrocyte/postive_cell_migration.score.in.time_2doamin.vlnplot.png",width = 4,height = 4,dpi = 300)

In [ ]:
da$time_dis_domain<-factor(da$time_dis_domain,levels = c('WT_sham-0.5mm-WM',
                                                 'WT_sham-1mm-WM',
                                                 'WT_3h-0.5mm-WM',
                                                 'WT_3h-1mm-WM',
                                                 'WT_24h-0.5mm-WM',
                                                 'WT_24h-1mm-WM',
                                                 'WT_72h-0.5mm-WM',
                                                 'WT_72h-1mm-WM',
                                                 'WT_sham-0.5mm-VH',
                                                 'WT_sham-1mm-VH',
                                                  'WT_3h-0.5mm-VH',
                                                 'WT_3h-1mm-VH',
                                                 'WT_24h-0.5mm-VH',
                                                 'WT_24h-1mm-VH',
                                                 'WT_72h-0.5mm-VH',
                                                 'WT_72h-1mm-VH',
                                                 'WT_sham-0.5mm-MG',
                                                 'WT_sham-1mm-MG',
                                                  'WT_3h-0.5mm-MG',
                                                 'WT_3h-1mm-MG',
                                                 'WT_24h-0.5mm-MG',
                                                 'WT_24h-1mm-MG',
                                                 'WT_72h-0.5mm-MG',
                                                 'WT_72h-1mm-MG',
                                                 'WT_sham-0.5mm-DH',
                                                 'WT_sham-1mm-DH',
                                                 'WT_3h-0.5mm-DH',
                                                 'WT_3h-1mm-DH',
                                                 'WT_24h-0.5mm-DH',
                                                 'WT_24h-1mm-DH',
                                                 'WT_72h-0.5mm-DH',
                                                 'WT_72h-1mm-DH' 
                                                ))
Idents(da)<-da$time_dis_domain
options(repr.plot.width=8,repr.plot.height=4)
VlnPlot(da,features = "Cluster1",
        pt.size = 0,
        cols = c(rep(c("#09DB36",'#20854EA8'),4),
                 rep(c("#F2E338",'#E18727A8'),4),
                 rep(c('#0D6DF5','#0072B5A8'),4),
                 rep(c('#F52C44','#BC3C29A8'),4)))+
    NoLegend()+
    geom_boxplot(width=0.35,fill="white",outlier.color = NA)
    #scale_color_manual(values = c(rep('#20854EA8',4),rep('#E18727A8',4),rep('#0072B5A8',4),rep('#BC3C29A8',4)))
ggsave("Astrocyte/postive_cell_migration.score.in.time_dis_4doamin.vlnplot.png",width = 8,height = 4,dpi = 300)

In [ ]:
da$time_dis_domain2<-factor(da$time_dis_domain2,levels = c('WT_sham-0.5mm-WM',
                                                 'WT_sham-1mm-WM',
                                                 'WT_3h-0.5mm-WM',
                                                 'WT_3h-1mm-WM',
                                                 'WT_24h-0.5mm-WM',
                                                 'WT_24h-1mm-WM',
                                                 'WT_72h-0.5mm-WM',
                                                 'WT_72h-1mm-WM',
                                                 'WT_sham-0.5mm-GM',
                                                 'WT_sham-1mm-GM',
                                                  'WT_3h-0.5mm-GM',
                                                 'WT_3h-1mm-GM',
                                                 'WT_24h-0.5mm-GM',
                                                 'WT_24h-1mm-GM',
                                                 'WT_72h-0.5mm-GM',
                                                 'WT_72h-1mm-GM'
                                                ))
Idents(da)<-da$time_dis_domain2
options(repr.plot.width=5,repr.plot.height=4)
VlnPlot(da,features = "Cluster1",
        pt.size = 0,
        cols = c(rep(c("#09DB36",'#20854EA8'),4),
                 #rep(c("#F2E338",'#E18727A8'),4),
                 rep(c('#0D6DF5','#0072B5A8'),4)#,
                 #rep(c('#F52C44','#BC3C29A8'),4)
                ))+
    NoLegend()+
    geom_boxplot(width=0.35,fill="white",outlier.color = NA)
    #scale_color_manual(values = c(rep('#20854EA8',4),rep('#E18727A8',4),rep('#0072B5A8',4),rep('#BC3C29A8',4)))
ggsave("Astrocyte/postive_cell_migration.score.in.time_dis_2doamin.vlnplot.png",width = 5,height = 4,dpi = 300)